In [ ]:
!pip install langchain
!pip install openai
!pip install langchain_openai

In [ ]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt_template = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
chat_model = ChatOpenAI()
output_parser = StrOutputParser() # Message의 content를 추출해서 string으로 변환

# 체인구성 prompt_template > chat_model
# 호출은 되지 않는다. 정의만 한다
chain = prompt_template | chat_model | output_parser

In [ ]:
chain

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me a short joke about {topic}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cdedef3ba30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cdedef38280>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [ ]:
# invoke 인자로 dict를 넘깁니다.
# 그 이유가 prompt_template이 dict를 받기 때문입니다.

chain.invoke({"topic": "ice cream"})

'Why did the ice cream truck break down? Because it had too many "scoops"!'

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt_template = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
chat_model = ChatOpenAI()
output_parser = StrOutputParser() # Message의 content를 추출해서 string으로 변환

# 체인구성 prompt_template > chat_model
# 호출은 되지 않는다. 정의만 한다
chain = {"topic": "ice cream"} | prompt_template | chat_model | output_parser

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

prompt_template = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
chat_model = ChatOpenAI()
output_parser = StrOutputParser()

chain = {"topic": RunnablePassthrough()} | prompt_template | chat_model | output_parser

In [ ]:
chain

{
  topic: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me a short joke about {topic}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cdedf0087f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cdedf0e84c0>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [ ]:
# invoke할 때, 굳이 topic 키로 dict로 넣어야 하나? 그냥 문자열로 넣으면 안될까?
# 문자열을 받아서, chain에 전달됨
# prompt_template의 입력 형식 dict로 받아야한다.
# 인자를 value로 dict를 구성해서 넘거야겠다.
# "ice cream" > RunnablePassthrough() > {"topic": RunnablePassthrough()} > prompt_template
chain.invoke("ice cream")

# RunnablePassthrough() = "ice cream"

# 질문 : RunnablePassthrough() 형식이 문자열인가요?

"Why did the ice cream truck break down? It couldn't handle the rocky road!"

In [ ]:
# 답 : invoke의 형식을 그대로 따릅니다. 만약 문자열이면 문자열, dict이면 dict입니다.

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

prompt_template = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
chat_model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

# 이렇게도 가능합니다.
chain = RunnablePassthrough() | prompt_template | chat_model | output_parser

chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get into arguments? \n\nBecause they always stay cool!"

In [ ]:
chain

RunnablePassthrough()
| ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me a short joke about {topic}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cdededfefe0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cdedefaa680>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [ ]:
RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),

(RunnableAssign(mapper={
   mult: RunnableLambda(lambda x: x['num'] * 3)
 }),)

In [ ]:
RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),

(RunnableAssign(mapper={
   mult: RunnableLambda(lambda x: x['num'] * 3)
 }),)

```
# object approach
chain = a.__or__(b)
chain("some input")

# pipe approach
chain = a | b
chain("some input")

```



In [ ]:
class Runnable:

    def __init__(self, func):
        self.func = func

    # 이 메서드는 파이썬의 비트 OR 연산자(|)를 오버로드합니다.
    def __or__(self, other):

        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            return other(self.func(*args, **kwargs))

        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [ ]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

In [ ]:
# wrap the functions with Runnable
add_five = Runnable(add_five)
multiply_by_two = Runnable(multiply_by_two)

In [ ]:
# chain the runnable functions together
chain = add_five | multiply_by_two

In [ ]:
# invoke the chain
chain(3)  # we should return 16

16

In [ ]:
# run them using the object approach
chain = add_five.__or__(multiply_by_two)
chain(3)  # should return 16

16

In [ ]:
def num2ko(x):
    return str(x) + "입니다."

In [ ]:
chain = add_five | multiply_by_two | num2ko

In [ ]:
chain(3)

# num2ko는 Runnable이 아닌데, 호출이 되나요?

'16입니다.'

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# 템플릿 만드는 방법

# 기획
# "Please generate dialogue sentences in English on the topic of health for a beginner level."
# "Please generate dialogue sentences in Korean on the topic of AI for an intermediate level."

# 양식과 변수를 분리하기
# "Please generate dialogue sentences in (English) on the topic of (health) for a (beginner) level."
# "Please generate dialogue sentences in (Korean) on the topic of (AI) for an (intermediate) level."

# "Please generate dialogue sentences in (language) on the topic of (topic) for a/an (level) level."

# 양식 조정하기
# "Please generate dialogue sentences in {language} on the topic of {topic} for a level of {level}."

In [ ]:
template = "Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}."

prompt_template = ChatPromptTemplate.from_template(template)

chain = prompt_template | chat_model # | StrOutputParser()

print(chain)

output = chain.invoke({"language" : "English", "topic" : "travel", "level" : "beginner"})

print(output)

first=ChatPromptTemplate(input_variables=['language', 'level', 'topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'level', 'topic'], template='Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}.'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cdededfefe0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cdedefaa680>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')
content="A: Hello, have you ever traveled abroad?\nB: No, I haven't but I would love to visit Italy one day.\nA: That sounds wonderful, Italy is known for its beautiful scenery and delicious food." response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 25, 'total_tokens': 69}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}


In [ ]:
# 변수 중에 시스템에서 입력해야할 것과 사용자가 입력해야할 것 분리
# language : 설정
# topic : 바뀐다.
# level : 설정

def get_learning_language(_):
    print("###")
    print(_)
    print("in get_learning_language")
    print("###")
    return "English"

def get_learning_level(_):
    print("###")
    print(_)
    print("in get_learning_level")
    print("###")
    return "beginner"

# 그럼 이 함수를 체인 내에서 어떻게 호출해야하는가?

In [ ]:
# 방법 1

template = "Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}."

prompt_template = ChatPromptTemplate.from_template(template)

chain = prompt_template | chat_model| StrOutputParser()

print(chain)

output = chain.invoke({"language" : get_learning_language(''), "topic" : "travel", "level" : get_learning_level('')})

print(output)

first=ChatPromptTemplate(input_variables=['language', 'level', 'topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'level', 'topic'], template='Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}.'))]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cdededfefe0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cdedefaa680>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')] last=StrOutputParser()
###

in get_learning_language
###
###

in get_learning_level
###
A: Have you ever traveled outside of your country?
B: No, I haven't. I have always wanted to explore new places.
A: Traveling is a great way to learn about different cultures and lifestyles.


In [ ]:
# 방법2

from langchain_core.runnables import RunnablePassthrough

template = "Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}."

prompt_template = ChatPromptTemplate.from_template(template)

chain = (
    RunnablePassthrough.assign(language = get_learning_language,
                               level = get_learning_level)
    | prompt_template
    | chat_model
)

# {"topic" : "travel"} > dict language, level 추가 > {"topic", ... "language", ... "level" ...} dict가 완성
# > prompt_template > chat_model

print(chain)

output = chain.invoke({"topic" : "travel"})

print(output)

# 넘어오는 인자에 추가적으로 key와 value를 넣고 싶을 때, RunnablePassthrough을 사용한다.

first=RunnableAssign(mapper={
  language: RunnableLambda(get_learning_language),
  level: RunnableLambda(get_learning_level)
}) middle=[ChatPromptTemplate(input_variables=['language', 'level', 'topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'level', 'topic'], template='Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}.'))])] last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cdedefc20b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cdedef39690>, openai_api_key=SecretStr('**********'), openai_proxy='')
###
{'topic': 'travel'}
in get_learning_language
###
###
{'topic': 'travel'}
in get_learning_level
###
content='1. "I want to go on a trip to a new country."\n2. "Where should I go for my next vacation?"\n3. "I love exploring new places and trying new foods when I travel."' response_metadata={'token_usage': {'completion_tok

In [ ]:
chain

RunnableAssign(mapper={
  language: RunnableLambda(get_learning_language),
  level: RunnableLambda(get_learning_level)
})
| ChatPromptTemplate(input_variables=['language', 'level', 'topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'level', 'topic'], template='Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}.'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cdedefc20b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cdedef39690>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [ ]:
from langchain_core.runnables import RunnablePassthrough

template = "Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}."

prompt_template = ChatPromptTemplate.from_template(template)

chain = (
    {"topic" : RunnablePassthrough()}
    | RunnablePassthrough.assign(language = get_learning_language,
                                 level = get_learning_level)

    | prompt_template
    | chat_model
)

print(chain)

output = chain.invoke("travel")

print(output)

first={
  topic: RunnablePassthrough()
} middle=[RunnableAssign(mapper={
  language: RunnableLambda(get_learning_language),
  level: RunnableLambda(get_learning_level)
}), ChatPromptTemplate(input_variables=['language', 'level', 'topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'level', 'topic'], template='Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}.'))])] last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cdededfefe0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cdedefaa680>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')
###
{'topic': 'travel'}
in get_learning_language
###
###
{'topic': 'travel'}
in get_learning_level
###
content='A: Do you like to travel?\nB: Yes, I love it. Where is your favorite place to travel?\nA: My favorite place is Paris, I love the culture and food there.' res

User
"언어 레벨에 따라 주어진 주제에 대해서 특정 언어 대화 문장 을 만들어줘."를 문장을 프롬프트에 맞게 다듬어줘.

2 / 2

ChatGPT
"언어 레벨에 따라 주어진 주제에 대한 특정 언어의 대화 문장을 생성해주세요."

User
"언어 레벨에 따라 주어진 주제에 대한 특정 언어의 대화 문장을 생성해주세요."

위 문장에 입력 인자를 "레벨", "주제", "언어"로 표시해

2 / 2

ChatGPT
"레벨에 따라 주제에 대한 언어의 대화 문장을 생성해주세요."

레벨: 사용자의 언어 능력 수준
주제: 대화의 주제
언어: 대화를 진행할 언어
User
"레벨에 따라 주제에 대한 언어의 대화 문장을 생성해주세요." 을 영어로
ChatGPT
"Please generate dialogue sentences in the specified language for the given topic, according to the language level."

User
"Please generate dialogue sentences in the specified language for the given topic, according to the language level."

위 문장을 아래 정보를 이용해서, 다시 수정해
- 쉬운 레벨, 건강, 영어
- 중간 레벨, AI, 한국어


2 / 2

ChatGPT
"Please generate dialogue sentences in English on the topic of health for a beginner level, and in Korean on the topic of AI for an intermediate level."

User
두 문장을 나누어
ChatGPT
"Please generate dialogue sentences in English on the topic of health for a beginner level."
"Please generate dialogue sentences in Korean on the topic of AI for an intermediate level."
User
"Please generate dialogue sentences in English on the topic of health for a beginner level."
"Please generate dialogue sentences in Korean on the topic of AI for an intermediate level."
"Please generate dialogue sentences in {language} on the topic of {topic} for an {level} level."

위에서 a {level} an {level}로 된다. 해결법은?
ChatGPT
For the phrase to grammatically adjust according to the article "a" or "an" before the word "level," based on whether the level starts with a vowel sound or not, you can use a conditional structure in your sentence. However, since we're working with a static text and not a dynamic language processing tool here, I'll provide a guideline on how to adjust it manually:

Use "an" before levels that start with a vowel sound (e.g., "intermediate" doesn't start with a vowel sound but "beginner" does not either; however, this rule is about sounds not just letters, so it's tricky without specific examples of vowel-starting levels like "advanced" where the "a" in "advanced" doesn't actually make it "an advanced" due to the consonant sound it starts with).
Use "a" before levels that start with a consonant sound.
Given this, the correct approach for each specific level needs to be determined by the sound of the first letter of the level descriptor (e.g., "an intermediate level" is incorrect because "intermediate" starts with a consonant sound, so it should be "a intermediate level", but normally "intermediate" would just use "a").

To universally apply this in a sentence template, you might rephrase to avoid the issue:

"Please generate dialogue sentences in {language} on the topic of {topic} for a level of {level}."

By structuring the sentence this way, you avoid the need to choose between "a" or "an" based on the level descriptor that follows.

User
# "Please generate dialogue sentences in {language} on the topic of {topic} for a level of {level}."

여기서 넣을 수 있는 변수 샘플 여러개
ChatGPT
Here are several examples of variables you can insert into the sentence "Please generate dialogue sentences in {language} on the topic of {topic} for a level of {level}.":

language: English, topic: travel, level: beginner

"Please generate dialogue sentences in English on the topic of travel for a level of beginner."
language: French, topic: cooking, level: intermediate

"Please generate dialogue sentences in French on the topic of cooking for a level of intermediate."
language: Spanish, topic: technology, level: advanced

"Please generate dialogue sentences in Spanish on the topic of technology for a level of advanced."
language: Korean, topic: culture, level: beginner

"Please generate dialogue sentences in Korean on the topic of culture for a level of beginner."
language: Japanese, topic: sports, level: intermediate

"Please generate dialogue sentences in Japanese on the topic of sports for a level of intermediate."
language: German, topic: environmental issues, level: advanced

"Please generate dialogue sentences in German on the topic of environmental issues for a level of advanced."
These examples cover a range of languages, topics, and proficiency levels to illustrate how you can customize the sentence template for various dialogue sentence generation requests.

User
외국어 연습을 하기 위해서는 3 sentences가 좋은가 4 sentences가 좋은가?
ChatGPT
외국어 연습을 위한 문장의 수는 학습자의 목표, 집중력, 그리고 시간에 따라 달라질 수 있습니다. 일반적으로, 연습을 위한 문장의 수는 다음과 같은 요소들을 고려하여 결정되어야 합니다:

학습 목표: 학습자가 단기간 내에 특정 주제나 어휘에 초점을 맞추려는 경우, 더 많은 문장을 연습하는 것이 도움이 될 수 있습니다. 반면, 특정 구조나 문법 규칙을 연습하는 데에는 적은 수의 문장으로도
User
dialogue 인 경우
ChatGPT
외국어 연습을 위한 대화(dialogue) 연습의 경우, 문장의 수는 연습의 목표와 상황에 따라 달라질 수 있습니다. 그러나 대화 스킬을 향상시키기 위해서는 다음과 같은 이유로 4개 문장이 3개 문장보다 더 유리할 수 있습니다:

대화의 흐름 이해: 4개의 문장은 3개의 문장보다 더 많은 정보와 대화의 전환을 제공하여 학습자가 대화의 흐름을 이해하는 데 도움을 줄 수 있습니다.
복합 대화 연습: 더 많은 문장을 사용하면 질문, 응답, 추가 설명, 반응 등 다양한 대화 요소를 연습할 수 있습니다. 이는 실제 대화에서 자연스러운 의사소통 능력을 키우는 데 유리합니다.
언어 다양성: 4개의 문장은 더 많은 어휘와 문법 구조를 사용하게 하여 언어 학습의 다양성과 깊이를 증가시킵니다.
상호작용 기회 증가: 대화에서 상호작용은 중요한 요소입니다. 4개의 문장을 사용함으로써 상대방과의 상호작용을 더 많이 연습할 수 있으며, 이는 실제 상황에서의 의사소통 능력 향상에 도움이 됩니다.
그러나 학습자의 수준과 특정 학습 목표에 따라 다를 수 있습니다. 초보자의 경우, 3개의 문장으로 시작하여 기본적인 대화를 연습하고, 점차 문장 수를 늘려가며 복잡한 대화 구조에 익숙해지는 것이 좋을 수 있습니다. 중급 이상의 학습자라면, 4개 이상의 문장을 포함한 더 복잡한 대화 연습이 더 적합할 수 있습니다.

User
template = "Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}."

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | chat_model

print(chain)

"language" : "English",
"level" : "beginner"

output = chain.invoke({"topic" : "travel"})

print(output)

여기서 language와 level는 시스템에서 수강 시에 미리 지정한 것을 사용하고자 한다. 즉 사용자가는 주제만 바꾸는 식이다. 그럼 시스템에서 지정된 것을 호출 하는 함수명으로 적합한 것은? language와 level를 따로 부른다.
ChatGPT
시스템에서 미리 지정된 language와 level 값을 호출하는 데 사용할 수 있는 함수명을 정의하려면, 명확하고 간결한 명명 규칙을 따르는 것이 좋습니다. 각각의 값을 따로 호출하는 경우, 함수명은 해당 값의 종류와 "get" 또는 "fetch"와 같은 동사를 결합하여 구성할 수 있습니다. 이러한 접근 방식은 함수의 목적을 명확히 전달하며, 어떤 종류의 값을 반환하는지 이해하기 쉽게 만듭니다.

예를 들어, language와 level 값을 호출하는 데 적합한 함수명은 다음과 같습니다:

getLanguage(): 현재 설정된 언어 값을 반환합니다.
getLevel(): 현재 설정된 레벨 값을 반환합니다.
이 함수들은 시스템에서 미리 지정된 값을 사용자나 다른 시스템 부분에 제공하는 역할을 합니다. 사용자는 주제만 선택하면, 이 함수들을 통해 현재 설정된 언어와 레벨에 따라 적절한 대화 문장을 생성할 수 있게 됩니다.

따라서, 시스템에서 지정된 language와 level를 호출하는 함수명으로 getLanguage()와 getLevel()이 적합합니다. 이 함수들은 각각 현재 설정된 언어와 레벨 값을 반환하여, 대화 문장 생성 과정에서 사용될 수 있습니다.

User
get_my_language 에서 내가 배우고자 하는 랭퀴지로 바꾸려면?
ChatGPT
내가 배우고자 하는 언어를 반환하는 함수의 이름을 지정할 때, 함수명은 그 기능을 명확하게 반영해야 합니다. get_my_language에서 "내가 배우고자 하는 언어"로 명확히 하려면, 함수명을 더 직관적으로 바꿀 수 있습니다. 예를 들어, 배우고자 하는 언어를 반환하는 함수의 경우, 다음과 같은 이름을 고려할 수 있습니다:

getTargetLanguage()
getLearningLanguage()
이러한 이름은 함수가 "배우고자 하는 언어"를 반환한다는 것을 더 명확하게 전달하며, 시스템 내에서 언어 학습 관련 설정이나 기능을 찾는 데 있어 직관적인 가이드를 제공합니다.



User
랭체인에서 템플릿안에 변수를 뭐라고 부르는가?
ChatGPT
랭체인(LangChain)이나 유사한 프로그래밍 환경에서 템플릿 안에 사용되는 변수를 일반적으로 "placeholder" 또는 "template variable"이라고 부릅니다. 이러한 변수는 템플릿 문자열에서 동적으로 교체될 값을 표시하기 위해 사용되며, 실행 시간에 실제 값으로 대체되어 최종 문자열을 생성합니다. Placeholder나 template variable은 템플릿을 더 유연하게 사용할 수 있게 해주며, 코드의 재사용성을 높여줍니다.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

def get_topic_language(input_situation):

    system_content_template="Please generate topic regarding a situation like {situation}"
    system_message_prompt_template = SystemMessagePromptTemplate.from_template(system_content_template)
    chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt_template])
    chat_model = ChatOpenAI(temperature=0)
    messages = chat_prompt_template.format_prompt(situation = input_situation['situation']).to_messages()
    response =  chat_model(messages)

    return response

In [ ]:
# 토픽을 주는 것이 아니라, 어떤 상황을 설명해주면 거기에 맞는 토픽을 자동으로 설정해준다.

from langchain_core.runnables import RunnablePassthrough

template = "Please generate dialogue three sentences in {language} on the topic of {topic} for a level of {level}."

prompt_template = ChatPromptTemplate.from_template(template)

chain = (
    RunnablePassthrough.assign(topic = get_topic_language,
                                 language = get_learning_language,
                                 level = get_learning_level)

    | prompt_template
    | chat_model
)

output = chain.invoke({"situation" : "온라인 강의하는 상황"})

print(output)

###
{'situation': '온라인 강의하는 상황'}
in get_learning_language
###
###
{'situation': '온라인 강의하는 상황'}
in get_learning_level
###
content='1. "Online learning can be both challenging and rewarding. It requires self-discipline but offers flexibility. Teachers may struggle to engage students but can reach a wider audience."\n\n2. "Staying motivated in online classes can be tricky. Setting personal goals can help, as can actively participating in discussions. Remember, it\'s about your growth, not just ticking off tasks!"\n\n3. "Technology has transformed education. It\'s not just about Zoom classes, it\'s about interactive learning platforms and a wealth of resources at our fingertips. But remember, tech is a tool, not a solution."\n\n4. "Creating a supportive online environment is crucial. We all need to feel comfortable and motivated to learn. This means respecting one another and making sure everyone feels included."\n\n5. "Managing time effectively is key in online learning. Prioritize tasks,

LLM기반 서비스 개발을 위한 프롬프트 만들기